In [2]:

def clean_csv_data(filepath):
    with open(filepath, "r") as file:
        lines = file.readlines()

    headers = lines[0].strip().split(",")
    cleaned_data = []

    for line in lines[1:]:
        values = line.strip().split(",")
        if len(values) != len(headers):
            continue

        row = dict(zip(headers, values))


        for key in row:
            val = row[key].strip().lower()
            if val in ["", "na", "null"]:
                row[key] = None


        for field in ["CLAIM_AMOUNT", "PREMIUM_COLLECTED", "PAID_AMOUNT"]:
            try:
                row[field] = float(row[field]) if row[field] is not None else 0.0
            except:
                row[field] = 0.0

        cleaned_data.append(row)

    return cleaned_data


data = clean_csv_data("/content/Insurance_auto_data.csv")
print(" Sample cleaned record:", data[0])


✅ Sample cleaned record: {'CLAIM_ID': 'CLM100021', 'CLAIM_DATE': '2025-04-01', 'CUSTOMER_ID': 'CUST14285', 'CLAIM_AMOUNT': 10419.0, 'PREMIUM_COLLECTED': 2198.59, 'PAID_AMOUNT': 6964.46, 'CITY': 'PUNE', 'REJECTION_REMARKS': None}


In [ ]:

def analyze_city_performance(cleaned_data):
    city_stats = {}

    for row in cleaned_data:
        city = row.get("CITY", "Unknown")
        city_stats.setdefault(city, {"claims": 0, "unpaid": 0.0})

        city_stats[city]["claims"] += 1

        unpaid = row["CLAIM_AMOUNT"] - row["PAID_AMOUNT"]
        if unpaid > 0:
            city_stats[city]["unpaid"] += unpaid

    for city, stats in city_stats.items():
        print(f"{city}: Claims = {stats['claims']}, Unpaid = ₹{stats['unpaid']}")

    worst_city = max(city_stats.items(), key=lambda x: x[1]["unpaid"])
    print("\n Recommendation: Close operations in:", worst_city[0])

analyze_city_performance(data)


PUNE: Claims = 37, Unpaid = ₹568698.11
GUWAHATI: Claims = 24, Unpaid = ₹451116.87999999995
RANCHI: Claims = 17, Unpaid = ₹298936.49
KOLKATA: Claims = 16, Unpaid = ₹234423.09000000003
None: Claims = 6, Unpaid = ₹139152.26

🔍 Recommendation: Close operations in: PUNE


In [ ]:

REJECTION_REASONS_MAP = {
    "fake_document": "Fake_document",
    "not_covered": "Not_Covered",
    "policy_expired": "Policy_expired"
}

def handle_error(error_message):
    print(f"Error: {error_message}")
    return "Error"

def contains_rejection_reason(rejection_text, reason):
    try:
        if rejection_text and isinstance(rejection_text, str):
            return reason in rejection_text.lower()
    except Exception as e:
        handle_error(f"Error in contains_rejection_reason: {str(e)}")
        return False
    return False

def map_rejection_reason(rejection_text):
    try:
        for reason, rejection_class in REJECTION_REASONS_MAP.items():
            if contains_rejection_reason(rejection_text, reason):
                return rejection_class
        return "Unknown"
    except Exception as e:
        handle_error(f"Error in map_rejection_reason: {str(e)}")
        return "Error"

def complex_rejection_classifier(remark_text):
    try:
        if not isinstance(remark_text, str) or len(remark_text.strip()) == 0:
            return "Invalid Remark"

        if contains_rejection_reason(remark_text, "fake_document"):
            return "Fake_document"
        elif contains_rejection_reason(remark_text, "not_covered"):
            return "Not_Covered"
        elif contains_rejection_reason(remark_text, "policy_expired"):
            return "Policy_expired"
        else:
            return map_rejection_reason(remark_text)
    except Exception as e:
        handle_error(f"Error in complex_rejection_classifier: {str(e)}")
        return "Error"

for record in data:
    remark = record.get("REJECTION_REMARKS")
    record["REJECTION_CLASS"] = complex_rejection_classifier(remark) if remark else "No Remark"

print(" Sample with rejection classification:", data[0])


✅ Sample with rejection classification: {'CLAIM_ID': 'CLM100021', 'CLAIM_DATE': '2025-04-01', 'CUSTOMER_ID': 'CUST14285', 'CLAIM_AMOUNT': 10419.0, 'PREMIUM_COLLECTED': 2198.59, 'PAID_AMOUNT': 6964.46, 'CITY': 'PUNE', 'REJECTION_REMARKS': None, 'REJECTION_CLASS': 'No Remark'}
